In [1]:
!pip install -U funasr

  Using cached funasr-1.2.6-py3-none-any.whl.metadata (32 kB)
Using cached funasr-1.2.6-py3-none-any.whl (701 kB)
  Attempting uninstall: funasr
    Found existing installation: funasr 1.0.27
    Uninstalling funasr-1.0.27:
      Successfully uninstalled funasr-1.0.27


In [9]:
from src.audio_function import diarize_audio, extract_audio_features,load_sensevoice, load_emotion2vec, audio_dict2description
from src.video2voice import extract_audio
from src.preprocess_extract import merge_rttm_intervals, split_media_files,extract_middle_frame
from tqdm import tqdm
import json
import os
import sys
from pathlib import Path
current_file = Path(os.getcwd()).resolve()
project_root = current_file.parent
sys.path.append(str(project_root / 'src'))
from get_body_data import analyze_person_image


In [2]:
text_model = load_sensevoice()
sar_model = load_emotion2vec()
print("模型加载完成")

funasr version: 1.2.6.


2025-04-15 23:04:43,409 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


Loading remote code failed: ./model.py, No module named 'model'


2025-04-15 23:04:46,549 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


funasr version: 1.2.6.


2025-04-15 23:04:48,344 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


Detect model requirements, begin to install it: C:\Users\19389\.cache\modelscope\hub\models\iic\emotion2vec_plus_large\requirements.txt
install model requirements successfully
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.0.0.weight, C:\Users\19389\.cache\modelscope\hub\models\iic\emotion2vec_plus_large\model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.0.0.bias, C:\Users\19389\.cache\modelscope\hub\models\iic\emotion2vec_plus_large\model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.1.0.weight, C:\Users\19389\.cache\modelscope\hub\models\iic\emotion2vec_plus_large\model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.1.0.bias, C:\Users\19389\.cache\modelscope\hub\models\iic\emotion2vec_plus_large\model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.2.0.weight, C:\Users\19389\.cache\modelscope\hub\models\iic\emotion2vec_plus_large\model.pt
Warning, miss key in ckpt: modality_en

In [10]:
def get_training_data(video_name):
    audio_dir = "../data/voice"
    video_dir = "../data/video"
    output_dir = "../data/output"  # 假设你有一个输出目录用于存放中间帧
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    # 构建路径
    video_path = os.path.join(video_dir, video_name)
    audio_path = os.path.join(audio_dir, video_name.replace('.mp4', '.wav'))
    if not os.path.exists(audio_path):
        extract_audio(video_path,'wav', audio_dir)
    frame_path = os.path.join(output_dir, video_name.replace('.mp4', '.jpg'))

    print(f"开始从视频 {video_path} 中提取音频...")
    # 提取音频特征
    print(f"正在分析 {audio_path} 的音频片段...")
    audio_features = extract_audio_features(audio_path, text_model, sar_model)
    audio_description = audio_dict2description(audio_features)

    # 提取视频中间帧并分析
    print(f"正在从视频 {video_path} 提取中间帧...")
    
    extract_middle_frame(video_path, frame_path)
    video_description = analyze_person_image(frame_path)

    # 组合描述
    combined_description = (
        f"Audio Analysis:\n{audio_description}\n\n"
        f"Video Analysis:\n{video_description}"
    )

    # 删除临时文件
    print("清理临时文件...")
    os.remove(frame_path)

    return combined_description




In [ ]:
result = []
import json
import os
# 使用 tqdm 包装 os.listdir 的结果
for i in tqdm(os.listdir('../data/video')[300:2300], desc="Processing videos"):
    if i.endswith('.mp4'):
        description = get_training_data(i)
        result.append({'video_name': i, 'description': description})
        os.system("cls")
if not os.path.exists('../data/output'):
    os.makedirs('../data/output')
# 保存结果
with open('../data/output/result.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)